In [ ]:
import pandas as pd

In [ ]:
X = pd.read_csv('pre_EF_data.csv')

In [ ]:
#Removing RFE columns
X_f = X.drop(columns = ['vdw_std','cov_rad_std','res_sg','z_std'],axis = 1)

In [ ]:
y_ef = pd.read_csv('y_EF.csv')

In [ ]:
#Shuffling the dataset for better learning by model
df_new = pd.concat([X_f, y_ef], axis=1)

df_new = df_new.sample(frac=1, random_state=42).reset_index(drop=True)

X_f = df_new.drop(columns = ['Ef'],axis = 1)
y_ef = df_new['Ef']

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_f, y_ef, test_size = 0.2, random_state = 1)

In [ ]:
from sklearn.preprocessing import StandardScaler
sc_X_f = StandardScaler()

X_train = sc_X_f.fit_transform(X_train)
X_test = sc_X_f.transform(X_test)

# **Keras Tuner for EF model**

In [ ]:
# import tensorflow as tf

In [ ]:
# !pip install keras_tuner

In [ ]:
# from tensorflow.keras.models import Sequential
# from tensorflow.keras.layers import Dense
# from tensorflow.keras.layers import Dropout


# import keras_tuner as kt
# def build_model_ef(hp):
#     model = Sequential()

#     # Adding the input layer
#     model.add(Dense(
#         units=hp.Int('units_input', min_value=150, max_value=200, step=50),
#         activation=hp.Choice('activation0',values = ['relu','tanh'])
#         input_dim=X_train.shape[1]
#     ))

#     model.add(Dropout(rate=hp.Float('dropout_input', min_value=0.1, max_value=0.5, step=0.1)))

#     # Adding intermediate layers
#     for i in range(hp.Int('num_layers', min_value=1, max_value=5)):
#         model.add(Dense(
#             units=hp.Int(f'units_{i}', min_value=75, max_value=150, step=25),
#             activation=hp.Choice(f'activation_{i}',values = ['relu','tanh'])
#         ))
#         model.add(Dropout(rate=hp.Float(f'dropout_{i}', min_value=0.1, max_value=0.5, step=0.1)))

#     # Adding the output layer
#     model.add(Dense(units=1))

#     model.compile(optimizer='adam', loss='mean_squared_error', metrics=['mean_squared_error'])

#     return model

# tuner_ef_arch = kt.RandomSearch(build_model_ef, objective='val_loss', max_trials=40, directory='mydir', project_name='ef_model')


In [ ]:
# tuner_ef_arch.search(X_train,y_train,batch_size = 32,epochs = 50 ,validation_data = (X_test , y_test))
# tuner_ef_arch.get_best_hyperparameters()[0].values

In [ ]:
# tuner_ef_arch.get_best_hyperparameters()[0].values

In [ ]:
# def build_model_ef_lr(hp):
#   model = tf.keras.models.Sequential()
#   model.add(tf.keras.layers.Dense(units=200, input_dim=X_train.shape[1], activation='relu'))
#   model.add(tf.keras.layers.Dropout(0.15))
#   model.add(tf.keras.layers.Dense(units=150, activation='relu'))
#   model.add(tf.keras.layers.Dropout(0.2))
#   model.add(tf.keras.layers.Dense(units=75, activation='tanh'))
#   model.add(tf.keras.layers.Dropout(0.1))
#   model.add(tf.keras.layers.Dense(units=1))

#   epoch = 150
#   batch_size = 32

#   initial_learning_rate = hp.Float('learning_rate',min_value = 1e-5,max_value = 1e-2,step = 1e-5)

#   optimizer = tf.keras.optimizers.Adam(
#     learning_rate = initial_learning_rate)

#   model.compile(loss='mean_squared_error',optimizer = optimizer, metrics='mean_squared_error')

#   return model

# tuner_ef_lr = kt.RandomSearch(build_model_ef_lr, objective='val_loss', max_trials=40, directory='mydir', project_name='ef_lr')
# tuner_ef_lr.search(X_train,y_train,batch_size = 32,epochs = 50 ,validation_data = (X_test , y_test))
# tuner_ef_lr.get_best_hyperparameters()[0].values


# **Ef Model with optimized parameters after kears tuner**

In [ ]:
import tensorflow as tf
model = tf.keras.models.Sequential()
model.add(tf.keras.layers.Dense(units=200, input_dim=X_train.shape[1], activation='relu'))
model.add(tf.keras.layers.Dropout(0.15))
model.add(tf.keras.layers.Dense(units=150, activation='relu'))
model.add(tf.keras.layers.Dropout(0.2))
model.add(tf.keras.layers.Dense(units=75, activation='tanh'))
model.add(tf.keras.layers.Dropout(0.1))
model.add(tf.keras.layers.Dense(units=1))

epochs = 150
batch_size = 32

initial_learning_rate = 0.0011 #As per the keras tuner

optimizer = tf.keras.optimizers.Adam(
    learning_rate = initial_learning_rate)

In [ ]:
from tensorflow.keras.callbacks import Callback
class LossHistory(Callback):
    def __init__(self):
        self.val_losses = []
        self.losses = []

    def on_epoch_end(self, epoch, logs=None):
        self.val_losses.append(logs.get('val_loss'))
        self.losses.append(logs.get('loss'))
history = LossHistory()

In [ ]:
from tensorflow.keras.callbacks import EarlyStopping

early_stopping = EarlyStopping(
    monitor='val_loss',
    patience=40,
    restore_best_weights=True
)

In [ ]:
from tensorflow.keras.callbacks import ModelCheckpoint
filepath = '/content/model_ef.h5'
checkpoint = ModelCheckpoint(filepath,
                             monitor='val_loss',
                             verbose=1,
                             save_best_only=True,
                             mode='min')

In [ ]:
ann.compile(loss='mean_squared_error',optimizer = optimizer, metrics='mean_squared_error')

In [ ]:
ann.fit(X_train, y_train, validation_data = (X_test,y_test) , batch_size = 32, epochs = 150,callbacks = [checkpoint,history,early_stopping])

In [ ]:
from tensorflow.keras.models import load_model
model = load_model('/content/model_ef.h5')

In [ ]:
from sklearn.metrics import mean_squared_error, r2_score , mean_absolute_error
import numpy as np
y_pred = model.predict(X_test)
mse = mean_squared_error(y_test, y_pred)
rmse = np.sqrt(mse)
r2 = r2_score(y_test, y_pred)
mean_abs = mean_absolute_error(y_test,y_pred)
print("Mean Squared Error:", mse)
print("Root Mean Squared Error:", rmse)
print("R-squared Score:", r2)
print("Mean Absolute Error:", mean_abs)

In [ ]:
val_losses = history.val_losses
loss = history.losses

In [ ]:
import matplotlib.pyplot as plt

epochs = list(range(1, 151))

plt.plot(epochs, val_losses, label='Val_Loss')
plt.plot(epochs, loss, label='Training Loss')
plt.title('Epoch vs Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()
plt.show()

In [ ]:
#Using predicted values of Ef by the model  , to augment into EG database
ef = model.predict(X_f)

# **Using K-Fold with optimized model hyperparameters**

In [ ]:
from sklearn.model_selection import KFold
kf = KFold(n_splits=5)

errors = []
kernels = []

for idx, (train, val) in enumerate(kf.split(X_f)):
    _X_cv_train = X_f.values[train]
    _X_cv_val = X_f.values[val]

    X_cv_train = scaler.fit_transform(_X_cv_train)
    X_cv_val = scaler.transform(_X_cv_val)

    y_cv_train = y_ef.values[train]
    y_cv_val = y_ef.values[val]

    model = tf.keras.models.Sequential()
    model.add(tf.keras.layers.Dense(units=200, input_dim=X_cv_train.shape[1], activation='relu'))
    model.add(tf.keras.layers.Dropout(0.15))
    model.add(tf.keras.layers.Dense(units=150, activation='relu'))
    model.add(tf.keras.layers.Dropout(0.2))
    model.add(tf.keras.layers.Dense(units=75, activation='tanh'))
    model.add(tf.keras.layers.Dropout(0.1))
    model.add(tf.keras.layers.Dense(units=1))

    epochs = 150
    batch_size = 32

    initial_learning_rate = 0.0011


    optimizer = tf.keras.optimizers.Adam(
        learning_rate = initial_learning_rate)

    model.compile(loss='mean_squared_error',optimizer = optimizer, metrics='mean_squared_error')
    model.fit(X_cv_train, y_cv_train, validation_data = (X_cv_val,y_cv_val) , batch_size = 32, epochs = 150,callbacks = [checkpoint])

    model = load_model('/content/model_ef.h5')

    y_pred_val = model.predict(X_cv_val)
    y_pred_train = model.predict(X_cv_train)

    # Computing errors
    rmse_val = np.sqrt(mean_squared_error(y_cv_val, y_pred_val))
    mse_val = mean_squared_error(y_cv_val, y_pred_val)
    rmse_train = np.sqrt(mean_squared_error(y_cv_train, y_pred_train))

    r2_val = r2_score(y_cv_val, y_pred_val)
    r2_train = r2_score(y_cv_train, y_pred_train)

    print("Root mean squared error: %.2f" % rmse_val)
    print("Coefficient of determination: %.2f" % r2_val)

    error = {'kfold': idx+1,
             'rmse_train': rmse_train,
             'r2_train': r2_train,
             'rmse_val': rmse_val,
             'r2_val': r2_val,
             'mse_val': mse_val
            }

    errors.append(error)

In [ ]:
errors_gpr_cv_opt_alpha = pd.DataFrame(errors)
errors_gpr_cv_opt_alpha.to_csv('ef_error_cv.csv',index = False)

In [ ]:
column_means = errors_gpr_cv_opt_alpha.mean()
print(column_means)

# **EG Model**

In [ ]:
#Removing RFE columns
X_g = pd.DataFrame(X.drop(columns =['spec_heat_mean','cs1','atom_mass_mean'],axis = 1))
X_g['Ef'] = ef

In [ ]:
y_eg = pd.read_csv('y_Eg.csv')

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_g, y_eg, test_size = 0.2, random_state = 1)

In [ ]:
from sklearn.preprocessing import StandardScaler
sc_X_g = StandardScaler()

X_train = sc_X_g.fit_transform(X_train)
X_test = sc_X_g.transform(X_test)

# **Keras Tuner for EG model**

In [ ]:

# def build_model_eg(hp):
#     model = Sequential()

#     # Adding the input layer
#     model.add(Dense(
#         units=hp.Int('units_input', min_value=150, max_value=200, step=50),
#         activation=hp.Choice('activation0',values = ['relu','tanh'])
#         input_dim=X_train.shape[1]
#     ))

#     model.add(Dropout(rate=hp.Float('dropout_input', min_value=0.1, max_value=0.5, step=0.1)))

#     # Adding intermediate layers
#     for i in range(hp.Int('num_layers', min_value=1, max_value=5)):
#         model.add(Dense(
#             units=hp.Int(f'units_{i}', min_value=75, max_value=150, step=25),
#             activation=hp.Choice(f'activation_{i}',values = ['relu','tanh'])
#         ))
#         model.add(Dropout(rate=hp.Float(f'dropout_{i}', min_value=0.1, max_value=0.5, step=0.1)))

#     # Adding the output layer
#     model.add(Dense(units=1))

#     model.compile(optimizer='adam', loss='mean_squared_error', metrics=['mean_squared_error'])

#     return model

# tuner_eg_arch = kt.RandomSearch(build_model_eg, objective='val_loss', max_trials=50, directory='mydir', project_name='eg_model')


In [ ]:
# tuner_eg_arch.search(X_train,y_train,batch_size = 32,epochs = 50 ,validation_data = (X_test , y_test))
# tuner_eg_arch.get_best_hyperparameters()[0].values

In [ ]:
# tuner_eg_arch.get_best_hyperparameters()[0].values

In [ ]:
# def build_model_eg_lr(hp):
  # model = tf.keras.models.Sequential()
  # model.add(tf.keras.layers.Dense(units=200, input_dim=X.shape[1], activation='tanh'))
  # model.add(tf.keras.layers.Dropout(0.1))
  # model.add(tf.keras.layers.Dense(units=100, activation='relu'))
  # model.add(tf.keras.layers.Dropout(0.2))
  # model.add(tf.keras.layers.Dense(units=100, activation='tanh'))
  # model.add(tf.keras.layers.Dropout(0.2))
  # model.add(tf.keras.layers.Dense(units=125, activation='tanh'))
  # model.add(tf.keras.layers.Dropout(0.5))
  # model.add(tf.keras.layers.Dense(units=150, activation='tanh'))
  # model.add(tf.keras.layers.Dropout(0.1))
  # model.add(tf.keras.layers.Dense(units=1))

#   epoch = 150
#   batch_size = 32

#   initial_learning_rate = hp.Float('learning_rate',min_value = 1e-5,max_value = 1e-2,step = 1e-5)

#   optimizer = tf.keras.optimizers.Adam(
#     learning_rate = initial_learning_rate)

#   model.compile(loss='mean_squared_error',optimizer = optimizer, metrics='mean_squared_error')

#   return model

# tuner_eg_lr = kt.RandomSearch(build_model_eg_lr, objective='val_loss', max_trials=40, directory='mydir', project_name='eg_lr')
# tuner_eg_lr.search(X_train,y_train,batch_size = 32,epochs = 50 ,validation_data = (X_test , y_test))
# tuner_eg_lr.get_best_hyperparameters()[0].values


# **Eg Model with optimized parameters after kears tuner**

In [ ]:
model = tf.keras.models.Sequential()
model.add(tf.keras.layers.Dense(units=200, input_dim=X.shape[1], activation='tanh'))
model.add(tf.keras.layers.Dropout(0.1))
model.add(tf.keras.layers.Dense(units=100, activation='relu'))
model.add(tf.keras.layers.Dropout(0.2))
model.add(tf.keras.layers.Dense(units=100, activation='tanh'))
model.add(tf.keras.layers.Dropout(0.2))
model.add(tf.keras.layers.Dense(units=125, activation='tanh'))
model.add(tf.keras.layers.Dropout(0.5))
model.add(tf.keras.layers.Dense(units=150, activation='tanh'))
model.add(tf.keras.layers.Dropout(0.1))
model.add(tf.keras.layers.Dense(units=1))

epochs = 150
batch_size = 32

initial_learning_rate = 0.0011

optimizer = tf.keras.optimizers.Adam(
    learning_rate = initial_learning_rate)

In [ ]:
filepath = '/content/model_eg.h5'
checkpoint = ModelCheckpoint(filepath,
                             monitor='val_loss',
                             verbose=1,
                             save_best_only=True,
                             mode='min')

In [ ]:
model.compile(loss='mean_squared_error',optimizer = optimizer, metrics='mean_squared_error')

In [ ]:
model.fit(X_train, y_train, validation_data = (X_test,y_test) , batch_size = batch_size, epochs = epochs,callbacks = [early_stopping,checkpoint,history])

In [ ]:
model = load_model('/content/model_eg.h5')

In [ ]:
evaluation = model.evaluate(X_test, y_test)

In [ ]:
y_pred = model.predict(X_test)
mse = mean_squared_error(y_test, y_pred)
rmse = np.sqrt(mse)
r2 = r2_score(y_test, y_pred)
mean_abs = mean_absolute_error(y_test,y_pred)
print("Mean Squared Error:", mse)
print("Root Mean Squared Error:", rmse)
print("R-squared Score:", r2)
print("Mean Absolute Error:", mean_abs)

In [ ]:
val_losses = history.val_losses
loss = history.losses

In [ ]:
import matplotlib.pyplot as plt

epochs = list(range(1, 151))

plt.plot(epochs, val_losses, label='Val_Loss')
plt.plot(epochs, loss, label='Training Loss')
plt.title('Epoch vs Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()
plt.show()

# **Using KFold**

In [ ]:
from sklearn.model_selection import KFold
kf = KFold(n_splits=5)

errors = []
kernels = []

for idx, (train, val) in enumerate(kf.split(X_g)):
    _X_cv_train = X_g.values[train]
    _X_cv_val = X_g.values[val]

    X_cv_train = scaler.fit_transform(_X_cv_train)
    X_cv_val = scaler.transform(_X_cv_val)

    y_cv_train = y_eg.values[train]
    y_cv_val = y_eg.values[val]

    model = tf.keras.models.Sequential()
    model.add(tf.keras.layers.Dense(units=200, input_dim=X_cv_train.shape[1], activation='tanh'))
    model.add(tf.keras.layers.Dropout(0.1))
    model.add(tf.keras.layers.Dense(units=100, activation='relu'))
    model.add(tf.keras.layers.Dropout(0.2))
    model.add(tf.keras.layers.Dense(units=100, activation='tanh'))
    model.add(tf.keras.layers.Dropout(0.2))
    model.add(tf.keras.layers.Dense(units=125, activation='tanh'))
    model.add(tf.keras.layers.Dropout(0.5))
    model.add(tf.keras.layers.Dense(units=150, activation='tanh'))
    model.add(tf.keras.layers.Dropout(0.1))
    model.add(tf.keras.layers.Dense(units=1))

    epochs = 150
    batch_size = 32
    eval_batch_size = 32

    train_data_size = X_cv_train.shape[0]
    steps_per_epoch = int(train_data_size / batch_size)
    num_train_steps = steps_per_epoch * epochs
    warmup_steps = int(0.25* num_train_steps)
    initial_learning_rate = 0.0011

    linear_decay = tf.keras.optimizers.schedules.PolynomialDecay(
        initial_learning_rate=initial_learning_rate,
        end_learning_rate=1e-4,
        decay_steps=num_train_steps)

    warmup_schedule = tf.keras.optimizers.schedules.PolynomialDecay(
        initial_learning_rate=initial_learning_rate,
        decay_steps=warmup_steps,
        end_learning_rate=initial_learning_rate,
        power=1.25
    )

    optimizer = tf.keras.optimizers.Adam(
        learning_rate = warmup_schedule)

    model.compile(loss='mean_squared_error',optimizer = optimizer, metrics='mean_squared_error')
    model.fit(X_cv_train, y_cv_train, validation_data = (X_cv_val,y_cv_val) , batch_size = 32, epochs = 150,callbacks = [checkpoint])

    model = load_model('/content/model_eg.h5')

    y_pred_val = model.predict(X_cv_val)
    y_pred_train = model.predict(X_cv_train)

    # Computing errors
    rmse_val = np.sqrt(mean_squared_error(y_cv_val, y_pred_val))
    mse_val = mean_squared_error(y_cv_val, y_pred_val)
    rmse_train = np.sqrt(mean_squared_error(y_cv_train, y_pred_train))

    r2_val = r2_score(y_cv_val, y_pred_val)
    r2_train = r2_score(y_cv_train, y_pred_train)

    print("Root mean squared error: %.2f" % rmse_val)
    print("Coefficient of determination: %.2f" % r2_val)

    error = {'kfold': idx+1,
             'rmse_train': rmse_train,
             'r2_train': r2_train,
             'rmse_val': rmse_val,
             'r2_val': r2_val,
             'mse_val': mse_val
            }

    errors.append(error)

In [ ]:
errors_gpr_cv_opt_alpha = pd.DataFrame(errors)
errors_gpr_cv_opt_alpha.to_csv('eg_cv.csv',index = False)

In [ ]:
column_means = errors_gpr_cv_opt_alpha.mean()
print(column_means)